# WebScraping

В этом скринкасте мы воспользуемся библиотекой *requests* для получения данных из сети;

используем библиотеку *Beautiful Soup* для работы с HTML ответами сервера;

и научимся работать с авторизацией в автоматическом режиме на простейшем примере.

# Информация

В контейнере на 5000 порту запущен сервис социальной сети (вы можете получить доступ к сервису по [URI `/app/`](/app/)), в которой пользователи могут только подписываться друг на друга, выстраивая некоторый социальный граф. Для каждого пользователя известны его id, имя, фамилия, подписчики и подписки.

#### Кроме того, известно, что некоторые пользователи сервиса закрыли свои данные от неавторизованных пользователей.

Спецификация API (для получения JSON ответа необходим заголовок `Accept: application/json`, иначе возвращается HTML):
* `/app/users/` - постраничная выдача всех пользователей, страницы задаются через URL параметр `page` (`/app/users?page=5`)
* `/app/users/<int:user_id>` - информация о конкретном пользователе

Кроме того, есть дополнительный ресурс `/protected/users/<int:user_id>`, который требует авторизации для всех запросов.

Примеры ответов API увидите ниже:

In [2]:
# запрос внутри контейнера, поэтому localhost
HOST = 'http://localhost:5000/app'
USERS = f'{HOST}/users'
USER = lambda user_id: f'{USERS}/{user_id}'
# тут все требуют авторизации
PROTECTED_USER = lambda user_id: f'{HOST}/protected/users/{user_id}'

In [3]:
import requests
import json
import base64

### HTML answer

In [4]:
html = requests.get(USER(8)).text

In [5]:
# первые 10 строк HTML ответа
print(*html.split('\n')[:10], sep='\n')

<!doctype html>
<html lang="en">
<head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <title>Hello, Mason! - WebScraping</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">


Для работы с HTML принято использовать библиотеку BeautifulSoup

Воспользуемся ей, чтобы обработать полученные данные

In [6]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(html)
# форматированный вывод
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Required meta tags -->
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <title>
   Hello, Mason! - WebScraping
  </title>
  <!-- Bootstrap CSS -->
  <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" rel="stylesheet"/>
 </head>
 <body>
  <nav class="navbar navbar-expand-lg navbar-light bg-light">
   <a class="navbar-brand" href="#">
    Hello, Mason!
   </a>
   <ul class="navbar-nav mr-auto">
    <li class="nav-item">
     <a class="nav-link" href="/app/">
      Hello World
     </a>
     <a class="nav-link" href="/jupyter">
      Go To Jupyter
     </a>
    </li>
   </ul>
  </nav>
  <section class="content">
   <header>
    <div class="jumbotron">
     <h1 class="display-4">
      Переход в
      <a href="/jupyter">
       Jupyter N

Попробуем найти все ссылки на других пользователей на странице,

* для поиска первого совпадения можно использовать метод `.find`, для всех — `.find_all`, `.select`;

* поиск происходит по дереву элементов HTML, в качестве поискового ключа указывается искомый тег

* также можно указывать дополнительные фильтры по аттрибутам.


Ссылки заключаются в тег `a`, аттрибут `href`, но по структуре файла мы видим,
что ссылки находятся внутри объекта `div` с `class="card-body"`, воспользуемся этим:

In [9]:
# зная, что ссылка лежит в div.card-body > a
links = [card.a['href'] for card in soup.find_all('div', attrs={'class': 'card-body'})]
# зная, что ссылка имеет вид `/app/users/\d`
links_href = [link['href'] for link in soup.select('a[href^="/app/users/"]')]

assert links == links_href
links

['/app/users/3782',
 '/app/users/713',
 '/app/users/8009',
 '/app/users/5579',
 '/app/users/973',
 '/app/users/4781',
 '/app/users/3026',
 '/app/users/9588',
 '/app/users/5974',
 '/app/users/8135',
 '/app/users/6857',
 '/app/users/9261',
 '/app/users/8750',
 '/app/users/7407',
 '/app/users/5492',
 '/app/users/2231',
 '/app/users/2460',
 '/app/users/6109',
 '/app/users/1566']

Теперь попробуем найти только тех, на кого пользователь подписан,

для этого воспользуемся тем, что подписки записаны в блок с заголовком Follows:

In [10]:
# DOM structure:
# |-div.row
# | |-div.col
# |   |-h2
# |     |-Follows:
# |-div.row
# | |-div.col
# |   |-...links...
follows = [l['href'] for l in soup.find('h2', text='Follows:').parent.parent.find_next_sibling('div').find_all('a')]
follows

['/app/users/8135',
 '/app/users/6857',
 '/app/users/9261',
 '/app/users/8750',
 '/app/users/7407',
 '/app/users/5492',
 '/app/users/2231',
 '/app/users/2460',
 '/app/users/6109',
 '/app/users/1566']

### JSON answer

Обратите внимание, мы используем тот же endpoint, но только с указанием заголовка Accept, указывая серверу, что мы ожидаем JSON; конечно, не все сервера гарантируют такое поведение.

In [11]:
response = requests.get(USER(1), headers={'Accept': 'application/json'})
json_data = json.loads(response.text)
assert json_data == response.json()

In [12]:
print(json.dumps(json_data, indent=4))

{
    "first_name": "Noah",
    "followers": [
        4672,
        3715,
        6885,
        7691,
        1136,
        4658,
        4086,
        5366,
        986
    ],
    "follows": [
        1604,
        4574,
        7678,
        7806,
        8554,
        3667,
        8819,
        3957,
        9943,
        6745,
        414
    ],
    "gravatar": "https://www.gravatar.com/avatar/c4ca4238a0b923820dcc509a6f75849b?s=150&d=identicon",
    "hash": "<built-in function openssl_md5>",
    "id": 1,
    "second_name": "Kelly"
}


## Использование авторизации

В примере используется Basic авторизация, представляющая из себя заголовок Authorization с содержанием "Basic %TOKEN%", где %TOKEN% - закодированная методом base64 пара "login:password". Это простейший механизм авторизации.

Удостоверимся в необходимости авторизации - ожидается статус `401 Unauthorized`:

In [13]:
err_resp = requests.get(PROTECTED_USER(1), 
                        headers={'Accept': 'application/json'})
print(err_resp)
assert err_resp.status_code == 401
print(err_resp.status_code)
print(err_resp.text)


<Response [401]>
401
Auth required


Пользователь с id=5 закрыл свой аккаунт и всегда требует авторизации, проверим:

In [14]:
err_resp = requests.get(USER(5), 
                        headers={'Accept': 'application/json'})
print(err_resp)
assert err_resp.status_code == 401
print(err_resp.status_code)
print(err_resp.text)


<Response [401]>
401
Auth required


Такой же ответ ожидается при использованиии неверного авторизационного токена:

In [15]:
token = 'wrong_token'
err_resp = requests.get(PROTECTED_USER(1), 
                        headers={'Accept': 'application/json',
                                 'Authorization': f"Basic {token}"})
print(err_resp)
print(err_resp.status_code)
print(err_resp.text)


<Response [401]>
401
Auth required


В данном случае, комбинация логина и пароля *login* и *password* является корректной, проверим это, используя схему basic авторизации (передаём в качестве токена строку состоящую из слова `Basic`, указывая на схему авторизации, и закодированной base64 пары `login:password`)

In [16]:
token = base64.standard_b64encode(b'login:password').decode('utf-8')

response = requests.get(PROTECTED_USER(1), 
                        headers={'Accept': 'application/json',
                                 'Authorization': f"Basic {token}"})

print(json.dumps(response.json(), indent=4))


{
    "first_name": "Noah",
    "followers": [
        4672,
        3715,
        6885,
        7691,
        1136,
        4658,
        4086,
        5366,
        986
    ],
    "follows": [
        1604,
        4574,
        7678,
        7806,
        8554,
        3667,
        8819,
        3957,
        9943,
        6745,
        414
    ],
    "gravatar": "https://www.gravatar.com/avatar/c4ca4238a0b923820dcc509a6f75849b?s=150&d=identicon",
    "hash": "<built-in function openssl_md5>",
    "id": 1,
    "second_name": "Kelly"
}
